In [3]:
import findspark

findspark.init()

import pyspark

In [4]:
pyspark

<module 'pyspark' from '/Users/shubhangigupta/hadoop/spark-3.3.0-bin-hadoop3/python/pyspark/__init__.py'>

In [5]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Word Count').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/23 10:48:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
ages_list = [21,23,24,25,26]

In [5]:
help(spark.createDataFrame)

Help on method createDataFrame in module pyspark.sql.session:

createDataFrame(data: Union[pyspark.rdd.RDD[Any], Iterable[Any], ForwardRef('PandasDataFrameLike')], schema: Union[pyspark.sql.types.AtomicType, pyspark.sql.types.StructType, str, NoneType] = None, samplingRatio: Optional[float] = None, verifySchema: bool = True) -> pyspark.sql.dataframe.DataFrame method of pyspark.sql.session.SparkSession instance
    Creates a :class:`DataFrame` from an :class:`RDD`, a list or a :class:`pandas.DataFrame`.
    
    When ``schema`` is a list of column names, the type of each column
    will be inferred from ``data``.
    
    When ``schema`` is ``None``, it will try to infer the schema (column names and types)
    from ``data``, which should be an RDD of either :class:`Row`,
    :class:`namedtuple`, or :class:`dict`.
    
    When ``schema`` is :class:`pyspark.sql.types.DataType` or a datatype string, it must match
    the real data, or an exception will be thrown at runtime. If the given s

##We can't create DataFrame with list of integers or Strings without infer schema or DataTypes or converting it into the tuples

In [7]:

 #spark.createDataFrame(ages_list)
spark.createDataFrame(ages_list,'int')

DataFrame[value: int]

In [8]:
from pyspark.sql.types import IntegerType
spark.createDataFrame(ages_list, IntegerType())

DataFrame[value: int]

In [11]:
names = ["ram","joe","zack","happy"]

In [14]:
df_string = spark.createDataFrame(names,'string')
df_string.show()

+-----+
|value|
+-----+
|  ram|
|  joe|
| zack|
|happy|
+-----+



In [16]:
from pyspark.sql.types import StringType
df2 = spark.createDataFrame(names,StringType())
df2.show()

+-----+
|value|
+-----+
|  ram|
|  joe|
| zack|
|happy|
+-----+



##WE CAN USE TUPLES TO CREATE SINGLE COLUMN DATAFRAME WITHOUT SPECIFYING THE SCHEMA

In [20]:
ages_list=[(21,),(22,),(23,),(24,)]

In [21]:
df = spark.createDataFrame(ages_list)
df.show()

+---+
| _1|
+---+
| 21|
| 22|
| 23|
| 24|
+---+



##But if we create tuple then specifying schema will be an issue

In [23]:
spark.createDataFrame(ages_list,'int')

TypeError: field value: IntegerType() can not accept object (21,) in type <class 'tuple'>

We can create the dataframe with tuple and datatype if we also provide column name

In [24]:
spark.createDataFrame(ages_list,' age int')

DataFrame[age: int]

In [7]:
user_list = [(1,'Scott'),(2,'Donald'), (3,'Mickey'), (4,'Bunny')]

In [9]:
df = spark.createDataFrame(user_list)
df.show()

+---+------+
| _1|    _2|
+---+------+
|  1| Scott|
|  2|Donald|
|  3|Mickey|
|  4| Bunny|
+---+------+



In [27]:
df = spark.createDataFrame(user_list,'user_id int, username string')
df.show()

+-------+--------+
|user_id|username|
+-------+--------+
|      1|   Scott|
|      2|  Donald|
|      3|  Mickey|
|      4|   Bunny|
+-------+--------+



Spark DataFrame is collection of row objects

In [31]:
# Collect will show a Dataframe into python list
df.collect()
type(df.collect())

list

In [32]:
from pyspark.sql import Row

In [33]:
help(Row) // Row(*args: Optional[str], **kwargs: Optional[Any]) -> 'Row'

Help on class Row in module pyspark.sql.types:

class Row(builtins.tuple)
 |  Row(*args: Optional[str], **kwargs: Optional[Any]) -> 'Row'
 |  
 |  A row in :class:`DataFrame`.
 |  The fields in it can be accessed:
 |  
 |  * like attributes (``row.key``)
 |  * like dictionary values (``row[key]``)
 |  
 |  ``key in row`` will search through row keys.
 |  
 |  Row can be used to create a row object by using named arguments.
 |  It is not allowed to omit a named argument to represent that the value is
 |  None or missing. This should be explicitly set to None in this case.
 |  
 |  .. versionchanged:: 3.0.0
 |      Rows created from named arguments no longer have
 |      field names sorted alphabetically and will be ordered in the position as
 |      entered.
 |  
 |  Examples
 |  --------
 |  >>> row = Row(name="Alice", age=11)
 |  >>> row
 |  Row(name='Alice', age=11)
 |  >>> row['name'], row['age']
 |  ('Alice', 11)
 |  >>> row.name, row.age
 |  ('Alice', 11)
 |  >>> 'name' in row
 | 

In [42]:
## Varying list of arguments
r=Row("Alice",11)
r


<Row('Alice', 11)>

In [43]:
## Varying keyword arguments
row2 = Row(name='Alice',age=11)
row2

Row(name='Alice', age=11)

In [44]:
row2.name

'Alice'

In [45]:
row2['name']

'Alice'

Convert List of Lists into SparkDataframe using Row

In [1]:
user_lists =[[1,'Scott'],[2,'Donald'], [3,'Mickey'], [4,'Bunny']]

In [11]:
df = spark.createDataFrame(user_lists)
df.show()

+---+------+
| _1|    _2|
+---+------+
|  1| Scott|
|  2|Donald|
|  3|Mickey|
|  4| Bunny|
+---+------+



In [12]:
df = spark.createDataFrame(user_lists, 'user_id int,user_name string')
df.show()

+-------+---------+
|user_id|user_name|
+-------+---------+
|      1|    Scott|
|      2|   Donald|
|      3|   Mickey|
|      4|    Bunny|
+-------+---------+



In [13]:
from pyspark.sql import Row

In [18]:
#converted list of lists into list of rows
# each list is of type row
user_rows = [Row(*user)for user in user_list]
user_rows
df = spark.createDataFrame(user_rows, 'user_id int,user_name string')
df.show()

+-------+---------+
|user_id|user_name|
+-------+---------+
|      1|    Scott|
|      2|   Donald|
|      3|   Mickey|
|      4|    Bunny|
+-------+---------+



In [19]:
def dummy(*args):
    print(args)

In [21]:
dummy(1,'hi')

(1, 'hi')


##when we have list and we want the list variables as varying arguments then we have to pass the value with * in the begning

In [20]:
# Passing as one arguments and giving one argument
user_details=[1,'Scott']
dummy(user_details)

([1, 'Scott'],)


In [22]:
# Passing as one arguments and giving varying argument
user_details=[1,'Scott']
dummy(*user_details)

(1, 'Scott')


##varying arguments *args and varying keyword arguments **kwargs
Row(*args, **kwargs)

In [ ]:
user_list = [(1,'scott'),(2,'Donald'),(3,'Mickey'),(4,'Elvis')]

In [24]:
df = spark.createDataFrame(user_list)
df.show()

+---+------+
| _1|    _2|
+---+------+
|  1| Scott|
|  2|Donald|
|  3|Mickey|
|  4| Bunny|
+---+------+



In [25]:
df = spark.createDataFrame(user_list, 'userid int, username string')
df.show()

+------+--------+
|userid|username|
+------+--------+
|     1|   Scott|
|     2|  Donald|
|     3|  Mickey|
|     4|   Bunny|
+------+--------+



In [26]:
from pyspark.sql import Row
def dummy(*args):
    print(args)
dummy(*user_details)

(1, 'Scott')


In [28]:
user_row = [Row(*user) for user in user_list]
type(user_row[0])

pyspark.sql.types.Row

In [31]:
df = spark.createDataFrame(user_row,'userid int, username string')
df.show()

+------+--------+
|userid|username|
+------+--------+
|     1|   Scott|
|     2|  Donald|
|     3|  Mickey|
|     4|   Bunny|
+------+--------+



In [35]:
user_list = [
    {'user_details':1, 'user_first_name':'scott'},
    {'user_details':2, 'user_first_name':'mickey'},
    {'user_details':3, 'user_first_name':'donald'},
    {'user_details':4, 'user_first_name':'bunny'},
]

In [36]:
user_details = user_list[0]
## .values gives list
user_details.values()
Row(*user_details.values())

dict_values([2, 'mickey'])

In [42]:
user_rows= [Row(*users.values()) for users in user_list]
df = spark.createDataFrame(user_rows)
df.show()

+---+------+
| _1|    _2|
+---+------+
|  1| scott|
|  2|mickey|
|  3|donald|
|  4| bunny|
+---+------+



In [43]:
## dict is converted to key word arguments by buiding the row object
user_rows= [Row(**users) for users in user_list]
df = spark.createDataFrame(user_rows)
df.show()

+------------+---------------+
|user_details|user_first_name|
+------------+---------------+
|           1|          scott|
|           2|         mickey|
|           3|         donald|
|           4|          bunny|
+------------+---------------+



In [50]:
def dummy(**kwargs):
    print(kwargs)
    print(len(kwargs))

In [51]:
user_details = {'user_id': 1, 'user_name':'scott'}
dummy(user_details=user_details)

{'user_details': {'user_id': 1, 'user_name': 'scott'}}
1


In [52]:
dummy(**user_details)

{'user_id': 1, 'user_name': 'scott'}
2


22/08/24 01:54:48 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 297830 ms exceeds timeout 120000 ms
22/08/24 01:54:48 WARN SparkContext: Killing executors is not supported by current scheduler.


## create dataframe with *args and **kwargs